In [60]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
import pandas as pd
import numpy as np
np.random.seed(0)

In [152]:
data = pd.read_csv("Cristano_Ronaldo_Final_v1/data.csv")
data = data.drop(data.columns[0],axis=1)
data = pd.get_dummies(data)
data.head()

,match_event_id,location_x,location_y,remaining_min,power_of_shot,knockout_match,remaining_sec,distance_of_shot,is_goal,shot_id_number,...,type_of_shot_shot - 6,type_of_shot_shot - 7,type_of_shot_shot - 8,type_of_shot_shot - 9,type_of_combined_shot_shot - 0,type_of_combined_shot_shot - 1,type_of_combined_shot_shot - 2,type_of_combined_shot_shot - 3,type_of_combined_shot_shot - 4,type_of_combined_shot_shot - 5
0,10.0,167.0,72.0,10.0,1.0,0.0,27.0,38.0,NaN,1.0,...,0,0,0,0,0,0,0,0,0,0
1,12.0,-157.0,0.0,10.0,1.0,0.0,22.0,35.0,0.0,2.0,...,0,0,0,0,0,0,0,0,0,0
2,35.0,-101.0,135.0,7.0,1.0,0.0,45.0,36.0,1.0,3.0,...,0,0,0,0,0,0,0,0,0,0
3,43.0,138.0,175.0,6.0,1.0,0.0,52.0,42.0,0.0,4.0,...,0,0,0,0,0,0,0,1,0,0
4,155.0,0.0,0.0,NaN,2.0,0.0,19.0,20.0,1.0,5.0,...,0,0,0,0,0,1,0,0,0,0


In [155]:
data = data.drop(['shot_id_number'],axis=1)
data.head()

,match_event_id,location_x,location_y,remaining_min,power_of_shot,knockout_match,remaining_sec,distance_of_shot,is_goal,match_id,...,type_of_shot_shot - 6,type_of_shot_shot - 7,type_of_shot_shot - 8,type_of_shot_shot - 9,type_of_combined_shot_shot - 0,type_of_combined_shot_shot - 1,type_of_combined_shot_shot - 2,type_of_combined_shot_shot - 3,type_of_combined_shot_shot - 4,type_of_combined_shot_shot - 5
0,10.0,167.0,72.0,10.0,1.0,0.0,27.0,38.0,NaN,20000012,...,0,0,0,0,0,0,0,0,0,0
1,12.0,-157.0,0.0,10.0,1.0,0.0,22.0,35.0,0.0,20000012,...,0,0,0,0,0,0,0,0,0,0
2,35.0,-101.0,135.0,7.0,1.0,0.0,45.0,36.0,1.0,20000012,...,0,0,0,0,0,0,0,0,0,0
3,43.0,138.0,175.0,6.0,1.0,0.0,52.0,42.0,0.0,20000012,...,0,0,0,0,0,0,0,1,0,0
4,155.0,0.0,0.0,NaN,2.0,0.0,19.0,20.0,1.0,20000012,...,0,0,0,0,0,1,0,0,0,0


In [156]:
#######LOCAL TRAIN-VALIDATION SPLIT########

#global train-test split:
train = data[data['is_goal'].notnull()]
y = train['is_goal']
test = data[data['is_goal'].isnull()]

# local training / validation performance:
train['local_train']=np.random.uniform(0,1,len(train))<=0.75
local_train, local_validation = train[train['local_train']==True], train[train['local_train']==False]
y_local_train = local_train['is_goal']
y_local_validation=local_validation['is_goal']
local_train=local_train.drop(['is_goal'],axis=1)
local_validation =local_validation.drop(['is_goal'],axis=1)

features = local_train.columns[:-1]
print(features)
local_train = local_train[features]
local_validation = local_validation[features]
print("local train size:", len(local_train))
print("local validation size:", len(local_validation))
print("test size:", len(test))

# preprocessing local-train+validation:
imp= SimpleImputer(missing_values=np.nan,strategy='mean')
imp = imp.fit(local_train)
local_train_modified = imp.transform(local_train)
imp = imp.fit(local_validation)
local_validation_modified = imp.transform(local_validation)

#RF classifier for train-validation perf:
clf = RandomForestClassifier(n_jobs=2, n_estimators=100, random_state=0)
clf.fit(local_train_modified, y_local_train)
y_validation_pred = clf.predict(local_validation_modified)
count_match = 0
count_error = 0

print("type,",type(y_validation_pred),type(y_local_validation))
assert(len(y_validation_pred)==len(y_local_validation))
validation_gtruth=np.asarray(y_local_validation)
for i in range(len(y_local_validation)):
    if (int(y_validation_pred[i])==int(validation_gtruth[i])):
        count_match+=1
    else:
        count_error+=1
validation_accuracy = count_match/(count_match+count_error)*100.0
print("validation a/c:", validation_accuracy)

/usr/lib/python3/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


Index(['match_event_id', 'location_x', 'location_y', 'remaining_min',
       'power_of_shot', 'knockout_match', 'remaining_sec', 'distance_of_shot',
       'match_id', 'team_id',
       ...
       'type_of_shot_shot - 6', 'type_of_shot_shot - 7',
       'type_of_shot_shot - 8', 'type_of_shot_shot - 9',
       'type_of_combined_shot_shot - 0', 'type_of_combined_shot_shot - 1',
       'type_of_combined_shot_shot - 2', 'type_of_combined_shot_shot - 3',
       'type_of_combined_shot_shot - 4', 'type_of_combined_shot_shot - 5'],
      dtype='object', length=1787)
local train size: 18236
local validation size: 6193
test size: 6268
type, <class 'numpy.ndarray'> <class 'pandas.core.series.Series'>
validation a/c: 61.747133860810585


In [157]:
#######ACTUAL TRAIN-TEST PERDICTION########

y_train = train['is_goal']
train=train.drop(['is_goal'],axis=1)
test =test.drop(['is_goal'],axis=1)
features = local_train.columns[:-1]
train = train[features]
test = test[features]
print("Train size:", len(train))
print("Test size:", len(test))

# preprocessing local-train+validation:
imp= SimpleImputer(missing_values=np.nan,strategy='mean')
imp = imp.fit(train)
train_modified = imp.transform(train)
imp = imp.fit(test)
test_modified = imp.transform(test)

#RF classifier for train-validation perf:
clf2 = RandomForestClassifier(n_jobs=2, n_estimators=100, random_state=0)
clf2.fit(train_modified, y_train)
y_test_pred = clf2.predict(test_modified)
count_match = 0
count_error = 0




# assert(len(y_test_pred)==len(y_local_validation))
# validation_gtruth=np.asarray(y_local_validation)
# for i in range(len(y_local_validation)):
#     if (int(y_validation_pred[i])==int(validation_gtruth[i])):
#         count_match+=1
#     else:
#         count_error+=1
# validation_accuracy = count_match/(count_match+count_error)*100.0
# print("validation a/c:", validation_accuracy)

Train size: 24429
Test size: 6268


In [158]:
#write outputs:
# shot_arr = np.asarray(test['shot_id_number'])
test_rows = data.index[data.is_goal.isnull()]
for i in range(len(test_rows)):
    print(str(int(test_rows[i]+1))+","+str(y_test_pred[i]))

1,0.0
8,0.0
17,1.0
20,1.0
22,0.0
33,0.0
34,1.0
35,1.0
36,1.0
37,1.0
38,0.0
45,0.0
50,0.0
55,0.0
60,0.0
62,0.0
66,0.0
67,0.0
71,0.0
72,0.0
76,0.0
80,0.0
85,0.0
86,0.0
87,0.0
92,0.0
95,1.0
97,0.0
104,0.0
113,0.0
122,1.0
123,0.0
126,1.0
133,1.0
134,0.0
141,0.0
144,0.0
150,0.0
152,0.0
153,0.0
156,0.0
159,1.0
165,1.0
172,0.0
182,1.0
192,0.0
193,0.0
196,0.0
202,0.0
205,0.0
210,0.0
211,1.0
228,1.0
250,1.0
257,0.0
260,1.0
262,0.0
264,0.0
267,0.0
279,1.0
282,0.0
293,0.0
299,1.0
300,0.0
301,0.0
317,0.0
327,0.0
328,0.0
335,0.0
346,1.0
348,0.0
360,0.0
365,0.0
379,0.0
382,0.0
390,1.0
391,1.0
392,0.0
397,0.0
417,0.0
418,0.0
420,0.0
428,0.0
429,0.0
434,1.0
439,1.0
440,0.0
451,0.0
456,0.0
459,0.0
472,0.0
480,0.0
491,0.0
497,0.0
509,0.0
511,0.0
523,0.0
533,1.0
544,0.0
546,0.0
552,0.0
555,1.0
558,0.0
560,0.0
568,0.0
571,1.0
573,0.0
576,1.0
578,1.0
591,0.0
596,0.0
597,1.0
601,1.0
605,0.0
617,0.0
625,0.0
628,0.0
634,0.0
646,0.0
649,0.0
651,1.0
654,1.0
656,0.0
657,1.0
661,1.0
669,0.0
670,1.0
680,1.0
693,0.

In [124]:
print(validation_gtruth)

[1. 1. 1. ... 0. 0. 0.]


In [142]:
print(len(test))

6268


In [147]:
test['shot_id_number']

0            1.0
7            8.0
16           NaN
19          20.0
21          22.0
32          33.0
33           NaN
34          35.0
35          36.0
36          37.0
37          38.0
44          45.0
49          50.0
54          55.0
59          60.0
61          62.0
65          66.0
66          67.0
70          71.0
71          72.0
75          76.0
79          80.0
84          85.0
85          86.0
86          87.0
91          92.0
94          95.0
96           NaN
103        104.0
112        113.0
          ...   
30567    30568.0
30569    30570.0
30580    30581.0
30583    30584.0
30590    30591.0
30593    30594.0
30613    30614.0
30616    30617.0
30617    30618.0
30625    30626.0
30629    30630.0
30630    30631.0
30631    30632.0
30633    30634.0
30635        NaN
30636    30637.0
30638    30639.0
30646    30647.0
30648    30649.0
30655    30656.0
30659    30660.0
30664    30665.0
30668    30669.0
30679        NaN
30680    30681.0
30681    30682.0
30682    30683.0
30686    30687

In [150]:
print(test_rows)

Int64Index([    0,     7,    16,    19,    21,    32,    33,    34,    35,
               36,
            ...
            30659, 30664, 30668, 30679, 30680, 30681, 30682, 30686, 30687,
            30693],
           dtype='int64', length=6268)
